In [1]:
#libs itialization
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
from prettytable import PrettyTable

In [2]:
Hu_code = {}
Hu_name = []
Hu_value = []

In [3]:
#Algorithm which searches the frequency of each english letter in the file
def findTheCharFrequency(text):
    freq = dict()
    with open(text,'r') as f:
        for line in f.readlines():
            line = line.lower()
            for i in line:
                if i.isalpha():
                    if i in freq:
                        freq[i] += 1
                    else:
                        freq.update({i:1})
    return freq
# sorted(l_frequency.items(), key=lambda item: item[1])

In [4]:
#Printing first algorithm result as a table of letter frequency
text = r'123.txt'
l_frequency = findTheCharFrequency(text)
th = []
td = []
for key, value in sorted(l_frequency.items()):
    th.append(key)
    td.append(value)
columns = len(th)
table = PrettyTable(th)
while td:
    table.add_row(td[:columns])
    td = th[columns:]
print(table)

+----+----+----+----+-----+----+----+---+-----+---+----+----+----+----+----+----+----+-----+-----+-----+----+---+---+
| a  | b  | c  | d  |  e  | f  | g  | h |  i  | k | l  | m  | n  | o  | p  | q  | r  |  s  |  t  |  u  | v  | w | y |
+----+----+----+----+-----+----+----+---+-----+---+----+----+----+----+----+----+----+-----+-----+-----+----+---+---+
| 98 | 11 | 52 | 49 | 147 | 12 | 19 | 8 | 141 | 1 | 83 | 56 | 66 | 57 | 33 | 19 | 67 | 108 | 101 | 110 | 34 | 3 | 1 |
+----+----+----+----+-----+----+----+---+-----+---+----+----+----+----+----+----+----+-----+-----+-----+----+---+---+


In [7]:
#Printing first algorithm result as a table of letter frequency
text = r'123.txt'
l_frequency = findTheCharFrequency(text)
th = []
td = []
for key, value in sorted(l_frequency.items(), key=lambda item: item[1]):
    th.append(key)
    td.append(value)
columns = len(th)
table = PrettyTable(th)
while td:
    table.add_row(td[:columns])
    td = th[columns:]
print(table)

+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+
| k | y | w | h | b  | f  | q  | g  | p  | v  | d  | c  | m  | o  | n  | r  | l  | a  |  t  |  s  |  u  |  i  |  e  |
+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+
| 1 | 1 | 3 | 8 | 11 | 12 | 19 | 19 | 33 | 34 | 49 | 52 | 56 | 57 | 66 | 67 | 83 | 98 | 101 | 108 | 110 | 141 | 147 |
+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+


In [5]:
# Создать класс узла
class Node(object):
    def __init__(self, name=None, value=None):
        self.name = name
        self.value = value
        self.lchild = None
        self.rchild = None

In [11]:
# Huffman tree creation
class HuffmanTree(object):
    # Based on node we built the tree in reverse algorithm
    def __init__(self, char_Weights):
        self.Leaf = [Node(k,v) for k, v in char_Weights.items()]
        while len(self.Leaf) != 1:
            self.Leaf.sort(key=lambda node:node.value, reverse=True)
            n = Node(value=(self.Leaf[-1].value + self.Leaf[-2].value))
            n.lchild = self.Leaf.pop(-1)
            n.rchild = self.Leaf.pop(-1)
            self.Leaf.append(n)
        self.root = self.Leaf[0]
        self.Buffer = list(range(10))

    def Hu_generate(self, tree, length):
        node = tree
        if (not node):
            return
        elif node.name:
            n = node.name
            b = str()
            for i in range(length):
                b += str(self.Buffer[i])
            Hu_code[n] = b
            Hu_name.append(n)
            Hu_value.append(b)
            return
        self.Buffer[length] = 0
        self.Hu_generate(node.lchild, length + 1)
        self.Buffer[length] = 1
        self.Hu_generate(node.rchild, length + 1)

    #Output part
    def get_code(self):
        self.Hu_generate(self.root, 0)
    

#Printing full algorithm result
tree = HuffmanTree(l_frequency)
tree.get_code()
fh = []
fd = []
#the reason of duplication of function is that this is another sorting algorithm 
for key, value in sorted(l_frequency.items(), key=lambda item: item[1]): #sorted(l_frequency.items()): - previous one
    fh.append(key)
    fd.append(value)

    
for i in range (0, len(fh)):
    z = fd[i]
    for j in range (0, len(fh)):
        x = Hu_name[j]
        print(x)
# columns = len(fh)
# print(fd)
# Hu_table = PrettyTable()
# for i in range(0, len(fh)):
#     Hu_table.add_column(fh[i], fd[i])
# print(Hu_table)


# print(fd)
print(Hu_code)



m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
a
d
c
t
s
u
m
o
y
k
w
h
g
p
n
i
r
v
q
b
f
e
l
